Note that you will need to use the **FlinkSql** kernel if you want to have autocompletion within SQL cells.

In [ ]:
%reload_ext jupytervvp
%connect_vvp -n default localhost -p 8080 -s mySession --force

In [ ]:
%%flink_sql
CREATE TABLE `testTable1` (id int)
COMMENT 'SomeComment'
WITH (
    'connector.type' = 'kafka',
    'connector.version' = 'universal',
    'connector.topic' = 'topic',
    'connector.properties.bootstrap.servers' = 'localhost',
    'format.type' = 'json',
    'format.derive-schema' = 'true'
)

In [ ]:
%%flink_sql
ALTER TABLE `testTable1` SET (
    'connector.type' = 'kafka',
    'connector.version' = 'universal',
    'connector.topic' = 'topic1',
    'connector.properties.bootstrap.servers' = 'localhost',
    'format.type' = 'json',
    'format.derive-schema' = 'true'
)

Variables can also be used:

In [ ]:
topic_name = "myTopic"
parallelism = 2
deployment_name = "testName"

In [ ]:
%%flink_sql
CREATE TABLE `testTable2` (id int)
COMMENT 'SomeComment'
WITH (
    'connector.type' = 'kafka',
    'connector.version' = 'universal',
    'connector.topic' = '{topic_name}',
    'connector.properties.bootstrap.servers' = 'localhost',
    'format.type' = 'json',
    'format.derive-schema' = 'true'
)

Deployment parameters (for e.g., `INSERT INTO` statements) can be set as a Python dictionary object.
Notice that variables here are not interpolated and usual Python syntax can be used:

In [ ]:
my_deployment_parameters = {
        "metadata.name": "{}".format(deployment_name),
        "spec.template.spec.parallelism": parallelism,
        "spec.upradeStrategy.kind": "STATEFUL",
        "spec.restoreStrategy.kind": "LATEST_STATE",
        "spec.template.spec.flinkConfiguration.state.backend": "filesystem",
        "spec.template.spec.flinkConfiguration.taskmanager.memory.managed.fraction": "0.1",
        "spec.template.spec.flinkConfiguration.high-availability": "vvp-kubernetes"
}

The following succeeds only when a default deployment target has been set up using the platform,
or when a target has been set up and is specified via a settings object.

In [ ]:
%%flink_sql -p my_deployment_parameters -d
INSERT INTO testTable2 SELECT * FROM testTable1

In [ ]:
%%flink_sql
DROP TABLE `testTable1`

In [ ]:
%%flink_sql
DROP TABLE `testTable2`

In [ ]:
%%flink_sql
SHOW TABLES